<a href="https://colab.research.google.com/github/ShubhamJadhav03/AIML/blob/main/bagging_Heart_Failure_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [3]:
dataset = pd.read_csv("/content/heart.csv")
dataset.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [12]:
dataset[dataset.Cholesterol < dataset.Cholesterol.mean()+3*dataset.Cholesterol.std()].shape

(915, 12)

In [13]:
df1 = dataset[dataset.Cholesterol < dataset.Cholesterol.mean()+3*dataset.Cholesterol.std()]

In [14]:
df2 = df1[df1.RestingBP < df1.RestingBP.mean()+3*df1.RestingBP.std()]

In [16]:
df2.shape

(908, 12)

In [17]:
df3 = df2[df2.MaxHR < df2.MaxHR.mean() + 3*df2.MaxHR.std()]

In [18]:
df3.shape

(908, 12)

In [19]:
df4 = df3.copy()

In [20]:
df4.RestingECG.unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [25]:
df1.ST_Slope.unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [26]:
df4['RestingECG'] = df4['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
})
df4['ExerciseAngina'] = df4['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
})
df4['ST_Slope'] = df4['ST_Slope'].replace({
    'Up': 0 ,
    'Flat': 1,
    'Down': 2
})
df4.head()

<ipython-input-26-3ab011ee7bef>:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df4['ST_Slope'] = df4['ST_Slope'].replace({


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,0,172,0,0.0,0,0
1,49,F,NAP,160,180,0,0,156,0,1.0,1,1
2,37,M,ATA,130,283,0,1,98,0,0.0,0,0
3,48,F,ASY,138,214,0,0,108,1,1.5,1,1
4,54,M,NAP,150,195,0,0,122,0,0.0,0,0


In [28]:
df4 = pd.get_dummies(df4, drop_first= True, dtype=int)
df4.head()

,Age,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA
0,40,140,289,0,0,172,0,0.0,0,0,1,1,0,0
1,49,160,180,0,0,156,0,1.0,1,1,0,0,1,0
2,37,130,283,0,1,98,0,0.0,0,0,1,1,0,0
3,48,138,214,0,0,108,1,1.5,1,1,0,0,0,0
4,54,150,195,0,0,122,0,0.0,0,0,1,0,1,0


In [29]:
X = df4.drop('HeartDisease', axis='columns')

In [30]:
y = df4.HeartDisease

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,stratify=y, test_size= 0.35, random_state = 30)

SVM Standalone Model

```
# This is formatted as code
```



In [36]:
from sklearn.svm import SVC

In [37]:
from sklearn.model_selection import cross_val_score

In [39]:
scores = cross_val_score(SVC(), X, y, cv=5)
scores.mean()

0.6893874081719384

Bagging + SVM

In [41]:
from sklearn.ensemble import BaggingClassifier
bag_model = BaggingClassifier(
    estimator= SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

0.6849553761155971

Standalone DecisionTreeClassifier

In [44]:
from sklearn.tree import DecisionTreeClassifier

In [46]:
scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores.mean()

0.7267439742577864

DecisionTreeClassifier + Bagging

In [45]:
bag_model = BaggingClassifier(
    estimator= DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
scores = cross_val_score(bag_model, X, y, cv=5)
scores.mean()

0.8027867160463845